# EP1 - Conjunto de Maldelbrot e paralelização com pThreads e OpenMP

| Nome | NUSP |
|------|------|
| Giulia C. de Nardi | 10692203 |
| Vitor D. Tamae | 10705620 |
| Lucy Anne de Omena Evangelista | 11221776 |
| Leonardo Costa Santos | 10783142 |
| Alexandre Muller Jones | 8038149 |


INFORMAÇÔES SOBRE O DESENVOLVIMENTO DO RELATÓRIO:

- **Para o caso dos mandelbrot paralelizados, podemos ter dois tempos como saída: o considerando o tempo de alocação de memória e I/O, e o tempo não considerando esses fatores. Ainda não implementado para a realização das funções, mas de fácil ajuste nas funções feitas neste notebook.**


## Configuração do ambiente

Atualizando os pacotes Julia

In [1]:
] up

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %Fetching: [==================>                      ]  45.0 % Resolving package versions...
 Installed Zlib_jll ────────── v1.2.11+10
 Installed Plots ───────────── v1.3.0
 Installed CategoricalArrays ─ v0.8.1
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
  [324d7699] ↑ CategoricalArrays v0.8.0 ⇒ v0.8.1
  [91a5bcdd] ↑ Plots v1.2.6 ⇒ v1.3.0
  [83775a58] ↑ Zlib_jll v1.2.11+9 ⇒ v1.2.11+10
  Building Plots → `~/.julia/packages/Plots/yuTb4/deps/build.log`


Verificando o status dos pacotes, e se há algum problema, com o comando:

In [2]:
] st

    Status `~/.julia/environments/v1.3/Project.toml`
  [336ed68f] CSV v0.6.2
  [a93c6f00] DataFrames v0.21.0
  [31c24e10] Distributions v0.23.2
  [7073ff75] IJulia v1.21.2
  [b964fa9f] LaTeXStrings v1.1.0
  [8314cec4] PGFPlotsX v1.2.6
  [1a8c2f83] Query v0.12.2
  [f3b207a7] StatsPlots v0.14.6


## Funções para realização dos experimentos

In [3]:
; make mandelbrot_seq

cc     mandelbrot_seq.c   -o mandelbrot_seq


In [4]:
; ./mandelbrot_seq

usage: ./mandelbrot_seq c_x_min c_x_max c_y_min c_y_max image_size
examples with image_size = 11500:
    Full Picture:         ./mandelbrot_seq -2.5 1.5 -2.0 2.0 11500
    Seahorse Valley:      ./mandelbrot_seq -0.8 -0.7 0.05 0.15 11500
    Elephant Valley:      ./mandelbrot_seq 0.175 0.375 -0.1 0.1 11500
    Triple Spiral Valley: ./mandelbrot_seq -0.188 -0.012 0.554 0.754 11500


In [5]:
; ./mandelbrot_seq 0.175 0.375 -0.1 0.1 200 0

0.041428


A função abaixo recebe parâmetros `size`, com o tamanho da imagem, `f`, com a id do tupo de fractal a ser feito (0 - Full, 2 - Triple Spiral, 3 - Elephant, 4 - Seahorse), `mandel`, com o nome da função a ser executada (`./mandelbrot_seq`, `./mandelbrot_opm`, `./mandelbrot_pth`),e `threads`, com o número de threads do programa paralelo. A função executa o programa `mandelbrot` com os parâmetros dados e devolve um `DataFrame` com os resultados.

In [6]:
using DataFrames, Query, StatsPlots, Statistics

function measure_mandelbrot(size, f, mandel; thread = 0)    
    if f == 0  mode = `-2.5 1.5 -2.0 2.0` #full
    elseif f == 1  mode = `-0.188 -0.012 0.554 0.754` #triple spiral
    elseif f == 2  mode = `0.175 0.375 -0.1 0.1` # elephant
    elseif f == 3  mode = `-0.8 -0.7 0.05 0.15` #seahorse
    end
    
    if thread != 0 
    results = parse.(Float64,
        read(`./$mandel $mode $size $thread`, String))
    else
    results = parse.(Float64,
        read(`./$mandel $mode $size`, String))
    end
        
    return DataFrame(size = size,
        f = f,
        threads = thread,
        duration = results[1])
end

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1273
┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1273


measure_mandelbrot (generic function with 1 method)

A função `run_experiments` recebe os mesmos parâmetros `size`, `f`,`mandel` e `threads`, e um parâmetro adicional `repetitions`, com o número de repetições de cada experimento com um dado número de `threads`. A função devolve um `DataFrame` com todos os experimentos.

In [7]:
function run_experiments(size, f, mandel, repetitions; threads = [])
    run(`make $mandel`)
    
    results = DataFrame(size = Int[],
        f = Int[],
        threads = Int[],
        duration = Float64[]) 
    
    if threads != []
    for t in threads
        for s in size
        for r in 1:repetitions
            append!(results,
                measure_mandelbrot(s, f, mandel, thread = t))    
        end
        end
    end
    else
        for r in 1:repetitions
        for s in size
            append!(results,
                measure_mandelbrot(s, f, mandel))    
        end
        end
    end 
    
    return results
end

run_experiments (generic function with 1 method)

A função `parse_results` recebe um `DataFrame` de resultados, produzido pela função `run_experiments`. A função devolve um `DataFrame` com a média e o intervalo de confiança da média a 95% dos tempos de execução, agrupados por número de threads.

In [8]:
function parse_results(results)
    parsed_results = results |>
                    @groupby({_.threads,_.size}) |>
                    @map({threads = key(_).threads,
                          size = _.size[1],
                          mean_duration = mean(_.duration),
                          ci_duration = 1.96 * std(_.duration)}) |>
                    DataFrame
    
    return parsed_results
end

parse_results (generic function with 1 method)

## Funções para traçar gráficos

A função abaixo permite que sejam traçadas até 5 séries de dados em um mesmo gráfico do tipo scatter.

In [9]:
pgfplotsx()

function plot_results(x, y, series_label, yerror; y2 = [], series_label2 = [], yerror2 = [], 
        y3 = [], series_label3 = [], yerror3 = [], y4 = [], series_label4 = [], yerror4 = [],
        y5 = [], series_label5 = [], yerror5 = [])
    max_thread_power = 5
    
    p = scatter(x, y, xaxis = :log2, xlabel = "Threads", xticks = [2 ^ x for x in 0:max_thread_power],
        yerror = yerror, alpha = 0.6, 
        labels = series_label, legend = :bottomright)
    
    if y2 != []
        p = scatter!(x, y2, xaxis = :log2, xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror2, alpha = 0.6,
            labels = series_label2, legend = :bottomright)
    end
    if y3 != []
        p = scatter!(x, y3, xaxis = :log2, xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror3, alpha = 0.6,
            labels = series_label3, legend = :bottomright)
    end
    if y4 != []
        p = scatter!(x, y4, xaxis = :log2, xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror4, alpha = 0.6,
            labels = series_label4, legend = :bottomright)
    end
    if y5 != []
        p = scatter!(x, y5, xaxis = :log2, xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror5, alpha = 0.6,
            labels = series_label5, legend = :bottomright)
    end
    
    return p
end

┌ Info: Precompiling PGFPlotsX [8314cec4-20b6-5062-9cdb-752b83310925]
└ @ Base loading.jl:1273
┌ Info: Precompiling Contour [d38c429a-6771-53c6-b99e-75d170b6e991]
└ @ Base loading.jl:1273


plot_results (generic function with 1 method)

## Condições para os experimentos

In [10]:
size = [2 ^ x for x in 4:13]
thread = [2 ^ x for x in 0:5]
repetitions = 10;

## Mandelbrot sequencial

Nesta parte, traremos a execução do maldelbrot em sua versão sequencial, junto com a análise de tempo de execução para os diferentes tipo de fractais (Triple Spiral, Elephant, Seahorse & Full) e em diferentes resoluções ($ 2^4 \cdots 2^{13}$)

Realizando as medições para o mandelbrot sequencial:

In [11]:
results = run_experiments(size, 0, "mandelbrot_seq", repetitions)
seq_full = parse_results(results)

make: 'mandelbrot_seq' is up to date.


,threads,size,mean_duration,ci_duration
,Int64,Int64,Float64,Float64
1,0,16,8.81e-5,3.21457e-5
2,0,32,0.0003463,0.000308654
3,0,64,0.0011882,0.000462979
4,0,128,0.0045518,0.00183678
5,0,256,0.0179811,0.00800325
6,0,512,0.0663697,0.0391323
7,0,1024,0.265033,0.16082
8,0,2048,0.776764,0.411441
9,0,4096,3.04184,1.38216


In [12]:
results = run_experiments(size, 1, "mandelbrot_seq", repetitions)
seq_triplespiral = parse_results(results);

make: 'mandelbrot_seq' is up to date.


ProcessFailedException: failed process: Process(`./mandelbrot_seq -0.188 -0.012 0.554 0.754 8192`, ProcessSignaled(2)) [0]


In [13]:
results = run_experiments(size, 2, "mandelbrot_seq", repetitions)
seq_elephant = parse_results(results);

make: 'mandelbrot_seq' is up to date.


ProcessFailedException: failed process: Process(`./mandelbrot_seq 0.175 0.375 -0.1 0.1 2048`, ProcessSignaled(2)) [0]


In [14]:
results = run_experiments(size, 3, "mandelbrot_seq", repetitions)
seq_seahorse = parse_results(results);

make: 'mandelbrot_seq' is up to date.


ProcessFailedException: failed process: Process(`./mandelbrot_seq -0.8 -0.7 0.05 0.15 2048`, ProcessSignaled(2)) [0]


## Mandelbrot com pthreads

In [31]:
results = run_experiments(size, 0, "mandelbrot_pth", repetitions,threads=thread)
pth_full = parse_results(results)

make: 'mandelbrot_pth' is up to date.


,threads,size,mean_duration,ci_duration
,Int64,Int64,Float64,Float64
1,1,16,0.0001323,3.90587e-5
2,1,32,0.0002998,8.53234e-5
3,1,64,0.0008287,3.88945e-5
4,1,128,0.0028507,0.000102571
5,1,256,0.0098278,0.00131417
6,1,512,0.038357,0.00219532
7,1,1024,0.144756,0.00586807
8,1,2048,0.796659,0.531016
9,1,4096,3.51485,1.18519


In [32]:
showall(pth_full)

60×4 DataFrame
│ Row │ threads │ size  │ mean_duration │ ci_duration │
│     │ Int64   │ Int64 │ Float64       │ Float64     │
├─────┼─────────┼───────┼───────────────┼─────────────┤
│ 1   │ 1       │ 16    │ 0.0001323     │ 3.90587e-5  │
│ 2   │ 1       │ 32    │ 0.0002998     │ 8.53234e-5  │
│ 3   │ 1       │ 64    │ 0.0008287     │ 3.88945e-5  │
│ 4   │ 1       │ 128   │ 0.0028507     │ 0.000102571 │
│ 5   │ 1       │ 256   │ 0.0098278     │ 0.00131417  │
│ 6   │ 1       │ 512   │ 0.038357      │ 0.00219532  │
│ 7   │ 1       │ 1024  │ 0.144756      │ 0.00586807  │
│ 8   │ 1       │ 2048  │ 0.796659      │ 0.531016    │
│ 9   │ 1       │ 4096  │ 3.51485       │ 1.18519     │
│ 10  │ 1       │ 8192  │ 12.5226       │ 1.90505     │
│ 11  │ 2       │ 16    │ 0.0001621     │ 5.11227e-5  │
│ 12  │ 2       │ 32    │ 0.0002224     │ 1.40185e-5  │
│ 13  │ 2       │ 64    │ 0.0004902     │ 2.68551e-5  │
│ 14  │ 2       │ 128   │ 0.0015714     │ 1.73644e-5  │
│ 15  │ 2       │ 256   │ 0.00574

In [16]:
results = run_experiments(size, 1, "mandelbrot_pth", repetitions,threads=thread)
pth_triplespiral = parse_results(results);

make: 'mandelbrot_pth' is up to date.


InterruptException: InterruptException:

In [17]:
results = run_experiments(size, 2, "mandelbrot_pth", repetitions,threads=thread)
pth_elephant = parse_results(results);

make: 'mandelbrot_pth' is up to date.


InterruptException: InterruptException:

In [18]:
results = run_experiments(size, 3, "mandelbrot_pth", repetitions,threads=thread)
pth_seahorse = parse_results(results);

make: 'mandelbrot_pth' is up to date.


InterruptException: InterruptException:

## Mandelbrot com OpenMP

In [30]:
results = run_experiments(size, 0, "mandelbrot_omp", repetitions,threads=thread)
omp_full = parse_results(results)

make: 'mandelbrot_omp' is up to date.


,threads,size,mean_duration,ci_duration
,Int64,Int64,Float64,Float64
1,1,16,8.34e-5,1.51032e-5
2,1,32,0.0002593,5.25034e-5
3,1,64,0.0007741,8.20991e-5
4,1,128,0.0027196,0.000378152
5,1,256,0.0098522,0.00121178
6,1,512,0.0364775,0.00131803
7,1,1024,0.208259,0.161465
8,1,2048,0.839414,0.696352
9,1,4096,3.54475,1.25549


In [20]:
results = run_experiments(size, 1, "mandelbrot_omp", repetitions,threads=thread)
omp_triplespiral = parse_results(results);

make: 'mandelbrot_omp' is up to date.


InterruptException: InterruptException:

In [21]:
results = run_experiments(size, 2, "mandelbrot_omp", repetitions,threads=thread)
omp_elephant = parse_results(results);

make: 'mandelbrot_omp' is up to date.


InterruptException: InterruptException:

In [22]:
results = run_experiments(size, 3, "mandelbrot_omp", repetitions,threads=thread)
omp_seahorse = parse_results(results);

make: 'mandelbrot_omp' is up to date.


InterruptException: InterruptException:

## Gráficos comparativos

Ao final, teremos os dataframes:

|Dataframe | Full | Triple Spiral | Seahorse |
|----------|--------|--------|--------|
|Sequencial|seq_full|seq_triplespiral|seq_seahorse|
|PThreads|pth_full|pth_triplespiral|pth_seahorse|
|OpenMP|omp_full|omp_triplespiral|omp_seahorse|

Realizaremos os gráficos a partir de partições do dataframe, como mostrados abaixo:

In [23]:
filter(row -> row[:threads] == 1, parsed_results)

UndefVarError: UndefVarError: parsed_results not defined

In [24]:
filter(row -> row[:size] == 16, parsed_results)

UndefVarError: UndefVarError: parsed_results not defined

In [25]:
filter(row -> row[:size] == 16, parsed_results).mean_duration

UndefVarError: UndefVarError: parsed_results not defined

### Comparando desempenho por tamanho da imagem

Ideias para os gráficos: 
> Comparar desempenho por tamanho da imagem ( 5 grafos > para tamanhos das imagens. Cada serie no grafico deve ser uma forma gerar a imagem)

> Comparar desempenho por tipo de gráfico produzido ( 3 x 4 gráficos, 4 áreas com 3 tipos de calculo cada / cruzar com tamanho da entrada também? daí seriam 4x 3 x 10)

> estou confouzer, me ajudem a saber quais gráficos fazer

In [26]:
plot_results(
    filter(row -> row[:size] == 16, parsed_results).threads,
    filter(row -> row[:size] == 16, parsed_results).mean_duration,    
    "16", filter(row -> row[:size] == 16, parsed_results).ci_duration,
    y2 = filter(row -> row[:size] == 32, parsed_results).mean_duration,
    series_label2 = "32", yerror2 = filter(row -> row[:size] == 32, parsed_results).ci_duration)

UndefVarError: UndefVarError: parsed_results not defined

### Comparando desempenho por quantidade de threads